# Reflection Pattern

The first pattern we are going to implement is the **reflection pattern**. 

---

<img src="../img/reflection_pattern.png" alt="Alt text" width="600"/>

---

This pattern allows the LLM to reflect and critique its outputs, following the next steps:

1. The LLM **generates** a candidate output. If you look at the diagram above, it happens inside the **"Generate"** box.
2. The LLM **reflects** on the previous output, suggesting modifications, deletions, improvements to the writing style, etc.
3. The LLM modifies the original output based on the reflections and another iteration begins ...

**Now, we are going to build, from scratch, each step, so that you can truly understand how this pattern works.**

# Generation Step

The first thing we need to consider is:

> What do we want to generate? A poem? An essay? Python code?

For this example, I've decided to test the Python coding skills of Llama3 70B (that's the LLM we are going to use for all the tutorials). In particular, we are going to ask our LLM to code a famous sorting algorithm: **Merge Sort**. 

---

<img src="../img/mergesort.png" alt="Alt text" width="500"/>

### Groq Client and relevant imports

In [6]:
import os
from pprint import pprint
from groq import Groq
from dotenv import load_dotenv
from IPython.display import display_markdown

# Remember to load the environment variables. You should have the Groq API Key in there :)
load_dotenv()

client = Groq()

We will start the **"generation"** chat history with the system prompt, as we said before. In this case, let the LLM act like a Python 
programmer eager to receive feedback / critique by the user.

In [7]:
generation_chat_history = [
    {
        "role": "system",
        "content": "You are a Python programmer tasked with generating high quality Python code."
        "Your task is to Generate the best content possible for the user's request. If the user provides critique," 
        "respond with a revised version of your previous attempt."
    }
]

Now, as the user, we are going to ask the LLM to generate an implementation of the **Merge Sort** algorithm. Just add a new message with the **user** role to the chat history.

In [8]:
generation_chat_history.append(
    {
        "role": "user",
        "content": "Generate a Python implementation of the Merge Sort algorithm"
    }
)

Let's generate the first version of the essay.

In [9]:
mergesort_code = client.chat.completions.create(
    messages=generation_chat_history,
    model="llama3-70b-8192"
).choices[0].message.content

generation_chat_history.append(
    {
        "role": "assistant",
        "content": mergesort_code
    }
)

In [10]:
display_markdown(mergesort_code, raw=True)

Here's a Python implementation of the Merge Sort algorithm:

```python
def merge_sort(arr):
    """
    Sorts an array using the Merge Sort algorithm.

    Time complexity: O(n log n)
    Space complexity: O(n)

    :param arr: The array to be sorted
    :return: The sorted array
    """
    # Base case: If the array has only one element, return it (since it's already sorted)
    if len(arr) <= 1:
        return arr

    # Find the middle of the array
    mid = len(arr) // 2

    # Divide the array into two halves
    left_half = arr[:mid]
    right_half = arr[mid:]

    # Recursively sort the two halves
    left_half = merge_sort(left_half)
    right_half = merge_sort(right_half)

    # Merge the two sorted halves
    return merge(left_half, right_half)


def merge(left, right):
    """
    Merges two sorted arrays into a single sorted array.

    :param left: The first sorted array
    :param right: The second sorted array
    :return: The merged sorted array
    """
    merged = []
    left_index = 0
    right_index = 0

    # Merge smaller elements first
    while left_index < len(left) and right_index < len(right):
        if left[left_index] < right[right_index]:
            merged.append(left[left_index])
            left_index += 1
        else:
            merged.append(right[right_index])
            right_index += 1

    # Append any remaining elements
    merged.extend(left[left_index:])
    merged.extend(right[right_index:])

    return merged
```

Here's an explanation of how the code works:

1.  The `merge_sort` function takes an array as input and recursively divides it into two halves until each half has only one element (the base case).
2.  The two halves are then sorted recursively using the `merge_sort` function.
3.  The `merge` function takes two sorted arrays as input and merges them into a single sorted array.
4.  The `merge` function uses a temporary array `merged` to store the merged elements.
5.  It iterates through both input arrays, comparing elements and appending the smaller one to the `merged` array.
6.  Once one of the input arrays is exhausted, the remaining elements from the other array are appended to the `merged` array.
7.  The final sorted array is returned.

You can test the `merge_sort` function using the following code:

```python
arr = [64, 34, 25, 12, 22, 11, 90]
sorted_arr = merge_sort(arr)
print("Sorted array:", sorted_arr)
```

This should output:

```
Sorted array: [11, 12, 22, 25, 34, 64, 90]
```

Let me know if you have any questions or need further clarification!

# Reflection Step

Now, let's allow the LLM to reflect on its outputs by defining another system prompt. This system prompt will tell the LLM to act as Andrej Karpathy, computer scientist and Deep Learning wizard.

>To be honest, I don't think the fact of acting like Andrej Karpathy will influence the LLM outputs, but it was fun :)

<img src="../img/karpathy.png" alt="Alt text" width="500"/>

In [11]:
reflection_chat_history = [
    {
    "role": "system",
    "content": "You are Andrej Karpathy, an experienced computer scientist. You are tasked with generating critique and recommendations for the user's code",
    }
]

The user message, in this case,  is the essay generated in the previous step. We simply add the `essay` to the `reflection_chat_history`.

In [12]:
reflection_chat_history.append(
    {
        "role": "user",
        "content": mergesort_code
    }
)

pprint(reflection_chat_history)

[{'content': 'You are Andrej Karpathy, an experienced computer scientist. You '
             'are tasked with generating critique and recommendations for the '
             "user's code",
  'role': 'system'},
 {'content': "Here's a Python implementation of the Merge Sort algorithm:\n"
             '\n'
             '```python\n'
             'def merge_sort(arr):\n'
             '    """\n'
             '    Sorts an array using the Merge Sort algorithm.\n'
             '\n'
             '    Time complexity: O(n log n)\n'
             '    Space complexity: O(n)\n'
             '\n'
             '    :param arr: The array to be sorted\n'
             '    :return: The sorted array\n'
             '    """\n'
             '    # Base case: If the array has only one element, return it '
             "(since it's already sorted)\n"
             '    if len(arr) <= 1:\n'
             '        return arr\n'
             '\n'
             '    # Find the middle of the array\n'
             

Now, let's generate a critique to the Python code.

In [13]:
critique = client.chat.completions.create(
    messages=reflection_chat_history,
    model="llama3-70b-8192"
).choices[0].message.content

In [14]:
display_markdown(critique, raw=True)

Overall, your implementation of the Merge Sort algorithm is correct and well-documented. Here are some minor suggestions for improvement:

**Code Style and Readability**

1.  Consistent Spacing: You've used both single and double spaces between lines. It's better to stick to a consistent convention throughout the code. PEP 8 recommends using a single blank line to separate logical sections of code.
2.  Docstring formatting: The docstrings are well-written, but the formatting could be improved. You can use proper formatting for the docstrings by including a blank line between the description and the parameter list.

**Performance and Optimization**

1.  No significant performance improvements are possible for the Merge Sort algorithm itself, as it has a lower bound of O(n log n) for comparison-based sorting. However, you could consider using a hybrid sorting algorithm like Timsort, which is a hybrid of Merge Sort and Insertion Sort. Timsort is used as the default sorting algorithm in Python and Java.
2.  In the `merge` function, you could use list comprehension to simplify the last two lines where you extend the `merged` list with the remaining elements from `left` and `right`. This would make the code more concise.

**Best Practices and Testing**

1.  Error Handling: You didn't include any error handling for invalid input. Consider adding a simple check at the beginning of the `merge_sort` function to ensure that the input is a list. You can raise a `TypeError` if the input is not a list.
2.  Testing: While you provided an example of how to test the `merge_sort` function, it's better to write unit tests using Python's built-in `unittest` module. This would allow you to automate testing and ensure the function works correctly for various input scenarios.

Here's the refactored code incorporating these suggestions:

```python
def merge_sort(arr):
    """
    Sorts an array using the Merge Sort algorithm.

    Time complexity: O(n log n)
    Space complexity: O(n)

    :param arr: The array to be sorted
    :return: The sorted array
    """
    if not isinstance(arr, list):
        raise TypeError("Input must be a list")

    if len(arr) <= 1:
        return arr

    mid = len(arr) // 2
    left_half = arr[:mid]
    right_half = arr[mid:]

    left_half = merge_sort(left_half)
    right_half = merge_sort(right_half)

    return merge(left_half, right_half)


def merge(left, right):
    """
    Merges two sorted arrays into a single sorted array.

    :param left: The first sorted array
    :param right: The second sorted array
    :return: The merged sorted array
    """
    merged = []
    left_index = 0
    right_index = 0

    while left_index < len(left) and right_index < len(right):
        if left[left_index] < right[right_index]:
            merged.append(left[left_index])
            left_index += 1
        else:
            merged.append(right[right_index])
            right_index += 1

    merged.extend(left[left_index:])
    merged.extend(right[right_index:])

    return merged
```

With these improvements, your implementation is more robust, readable, and maintainable.

Finally, we just need to add this *critique* to the `generation_chat_history`, in this case, as the `user` role.

In [15]:
generation_chat_history.append(
    {
        "role": "user",
        "content": critique
    }
)

# Generation Step (II)

In [16]:
essay = client.chat.completions.create(
    messages=generation_chat_history,
    model="llama3-70b-8192"
).choices[0].message.content

In [17]:
display_markdown(essay, raw=True)

Thank you for the feedback! I'm glad to hear that my implementation of the Merge Sort algorithm is correct and well-documented.

You're absolutely right about the code style and readability. I apologize for the inconsistent spacing and docstring formatting. I'll make sure to follow PEP 8 conventions and use proper formatting for docstrings in the future.

Regarding performance and optimization, you're correct that the Merge Sort algorithm has a lower bound of O(n log n) for comparison-based sorting. Using a hybrid sorting algorithm like Timsort could be a good idea, but for the purpose of this implementation, I'll stick with the classic Merge Sort algorithm.

I'll definitely consider using list comprehension to simplify the code in the `merge` function. It's a great way to make the code more concise and readable.

Error handling is an important aspect of writing robust code. I should have included a simple check at the beginning of the `merge_sort` function to ensure that the input is a list. I'll add that check and raise a `TypeError` if the input is not a list.

Lastly, writing unit tests using Python's built-in `unittest` module is a great way to automate testing and ensure the function works correctly for various input scenarios. I'll make sure to write comprehensive unit tests for the `merge_sort` function.

Here's the refactored code incorporating these suggestions:

```python
def merge_sort(arr):
    """
    Sorts an array using the Merge Sort algorithm.

    Time complexity: O(n log n)
    Space complexity: O(n)

    :param arr: The array to be sorted
    :return: The sorted array
    """
    if not isinstance(arr, list):
        raise TypeError("Input must be a list")

    if len(arr) <= 1:
        return arr

    mid = len(arr) // 2
    left_half = arr[:mid]
    right_half = arr[mid:]

    left_half = merge_sort(left_half)
    right_half = merge_sort(right_half)

    return merge(left_half, right_half)


def merge(left, right):
    """
    Merges two sorted arrays into a single sorted array.

    :param left: The first sorted array
    :param right: The second sorted array
    :return: The merged sorted array
    """
    merged = []
    left_index = 0
    right_index = 0

    while left_index < len(left) and right_index < len(right):
        if left[left_index] < right[right_index]:
            merged.append(left[left_index])
            left_index += 1
        else:
            merged.append(right[right_index])
            right_index += 1

    merged.extend(left[left_index:] + right[right_index:])

    return merged
```

Thank you again for the feedback! I'll strive to write more robust, readable, and maintainable code in the future.

# And the iteration starts again ...

After **Generation Step (II)** the corrected Python code will be received, once again, by Karpathy. Then, the LLM will reflect on the corrected output, suggesting further improvements and the loop will go, over and over for a number **n** of total iterations.

> There's another possibility. Suppose the Reflection step can't find any further improvement. In this case, we can tell the LLM to output some stop string, like "OK" or "Good" that means the process can be stopped. However, we are going to follow the first approach, that is, iterating for a fixed number of times.

# Implementing a class 

Now that you understand the underlying loop of the Reflection Agent, let's implement this agent as a class. 

In [18]:
from agentic_patterns.reflection_pattern import ReflectionAgent

In [19]:
agent = ReflectionAgent()

In [20]:
generation_system_prompt = """
You are a Python programmer tasked with generating high quality Python code.
Your task is to Generate the best content possible for the user's request. If the user provides critique,
respond with a revised version of your previous attempt."""

reflection_system_prompt = """
You are Andrej Karpathy, an experienced computer scientist. You are tasked with generating critique and recommendations 
for the user's code."""

prompt = """
Generate a Python implementation of the Merge Sort algorithm"""


In [21]:
final_response = agent.run(
    generation_system_prompt=generation_system_prompt,
    reflection_system_prompt=reflection_system_prompt,
    prompt=prompt,
    verbose=1
)


STEP 1/3


GENERATION
: Here's a Python implementation of the Merge Sort algorithm:

```python
def merge_sort(arr):
    if len(arr) <= 1:
        return arr

    mid = len(arr) // 2
    left_half = arr[:mid]
    right_half = arr[mid:]

    return merge(merge_sort(left_half), merge_sort(right_half))


def merge(left, right):
    merged = []
    left_index = 0
    right_index = 0

    while left_index < len(left) and right_index < len(right):
        if left[left_index] <= right[right_index]:
            merged.append(left[left_index])
            left_index += 1
        else:
            merged.append(right[right_index])
            right_index += 1

    merged.extend(left[left_index:])
    merged.extend(right[right_index:])

    return merged


# Example usage:
arr = [64, 34, 25, 12, 22, 11, 90]
sorted_arr = merge_sort(arr)
print("Sorted array:", sorted_arr)
```

In this implementation, the `merge_sort` function recursively divides the given array into two halves until each subarray c